In [158]:
from sqlalchemy import create_engine, MetaData
import json
import sys
from datetime import datetime
from collections import defaultdict

In [157]:
engine = create_engine('postgresql://postgres@localhost:5432/postgres', echo=True)
con = engine.connect()

2018-10-03 21:18:56,618 INFO sqlalchemy.engine.base.Engine select version()
2018-10-03 21:18:56,620 INFO sqlalchemy.engine.base.Engine {}
2018-10-03 21:18:56,624 INFO sqlalchemy.engine.base.Engine select current_schema()
2018-10-03 21:18:56,627 INFO sqlalchemy.engine.base.Engine {}
2018-10-03 21:18:56,636 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-10-03 21:18:56,638 INFO sqlalchemy.engine.base.Engine {}
2018-10-03 21:18:56,643 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-10-03 21:18:56,645 INFO sqlalchemy.engine.base.Engine {}
2018-10-03 21:18:56,649 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2018-10-03 21:18:56,651 INFO sqlalchemy.engine.base.Engine {}


## Load waze data from json

In [141]:
data = json.load(open('data/raw/Lima_2018-10-03_16:29:03.json', 'r'))

In [142]:
data.keys()

dict_keys(['alerts', 'endTimeMillis', 'irregularities', 'startTimeMillis', 'startTime', 'endTime', 'jams'])

In [152]:
def to_default_dict(list_of_dic):
    return [defaultdict(lambda: None, dic) for dic in list_of_dic]

jams, alerts, irregularities = list(map(to_default_dict, 
                                        [data['jams'], data['alerts'], data['irregularities']]))

## Load Schema from DB

In [132]:
metadata = MetaData()
metadata.reflect(engine, schema='waze')

In [134]:
metadata.tables['waze.jams']

Table('jams', MetaData(bind=None), Column('id', INTEGER(), table=<jams>, primary_key=True, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x11388ae80>, for_update=False)), Column('uuid', TEXT(), table=<jams>, nullable=False), Column('pub_millis', BIGINT(), table=<jams>, nullable=False), Column('pub_utc_date', TIMESTAMP(), table=<jams>), Column('start_node', TEXT(), table=<jams>), Column('end_node', TEXT(), table=<jams>), Column('road_type', SMALLINT(), table=<jams>), Column('street', TEXT(), table=<jams>), Column('city', TEXT(), table=<jams>), Column('country', TEXT(), table=<jams>), Column('delay', SMALLINT(), table=<jams>), Column('speed', REAL(), table=<jams>), Column('speed_kmh', REAL(), table=<jams>), Column('length', SMALLINT(), table=<jams>), Column('turn_type', TEXT(), table=<jams>), Column('level', SMALLINT(), table=<jams>), Column('blocking_alert_id', TEXT(), table=<jams>), Column('line', JSON(astext_type=Text()), table=<jams>), Col

## Insert jams to DB

In [153]:
def make_jam_dict(jam, data):
    return dict(uuid=jam['uuid'],
                pub_millis=jam['pubMillis'],
                pub_utc_date=datetime.strptime(data['startTime'], '%Y-%m-%d %H:%M:%S:%f'),
                start_node=jam['startNode'],
                end_node=jam['endNode'],
                road_type=jam['roadType'],
                street=jam['street'],
                city=jam['city'],
                country=jam['country'],
                delay=jam['delay'],
                speed=jam['speed'],
                speed_kmh=jam['speedKMH'],
                length=jam['length'],
                turn_type=jam['turnType'],
                level=jam['level'],
                blocking_alert_id=jam['blockingAlertId'],
                line=jam['line'],
                type=jam['type'],
                turn_line=jam['turnType'])

In [154]:
with engine.connect() as conn:
    for jam in jams:
        conn.execute(metadata.tables['waze.jams'].insert(), make_jam_dict(jam, data))

2018-10-03 21:16:21,159 INFO sqlalchemy.engine.base.Engine INSERT INTO waze.jams (uuid, pub_millis, pub_utc_date, start_node, end_node, road_type, street, city, country, delay, speed, speed_kmh, length, turn_type, level, blocking_alert_id, line, type, turn_line) VALUES (%(uuid)s, %(pub_millis)s, %(pub_utc_date)s, %(start_node)s, %(end_node)s, %(road_type)s, %(street)s, %(city)s, %(country)s, %(delay)s, %(speed)s, %(speed_kmh)s, %(length)s, %(turn_type)s, %(level)s, %(blocking_alert_id)s, %(line)s, %(type)s, %(turn_line)s) RETURNING waze.jams.id
2018-10-03 21:16:21,163 INFO sqlalchemy.engine.base.Engine {'uuid': 1265563837, 'pub_millis': 1538587878938, 'pub_utc_date': datetime.datetime(2018, 10, 3, 19, 26), 'start_node': None, 'end_node': 'Av. Ariosto Matellini', 'road_type': 2, 'street': 'Av. Defensores del Morro', 'city': 'Chorrillos', 'country': 'PE', 'delay': 138, 'speed': 4.097222222222222, 'speed_kmh': 14.75, 'length': 784, 'turn_type': 'NONE', 'level': 3, 'blocking_alert_id': Non

In [159]:
from pathlib import Path

In [167]:
p = Path('.').parent / 'data' / 'raw'

In [170]:
a = list(p.glob('*.json'))[0]

In [172]:
import os

In [178]:
os.rename(a, p.parent / 'processed' / a.name)